In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
word_df=pd.read_csv('5-letter.csv', index_col=0)
print(word_df.head())
word_list = word_df.word.iloc[0:5000,].to_list()
#print(word_list)
print(len(word_list))


     word       count
25  WHICH  3140226612
33  THEIR  2152980325
40  THERE  1623700147
46  WOULD  1472811049
51  OTHER  1383185827
5000


In [3]:
np.where(word_df.word=="SUBSP")

(array([5403], dtype=int64),)

In [283]:
class Wordle:
    def __init__(self, word=None, n_guesses=6, n_letters=5, all_words=['PROOF','SHEAR','BUMPY']):
        if word == None:
            self.word = random.sample(set(all_words), 1)[0]
        else:
            self.word = word.upper()
        self.n_guesses = n_guesses
        self.n_letters = n_letters
        self.guess_counter = n_guesses
        self.greens = [None] * self.n_letters 
        self.current_yellows = [None] * self.n_letters
        self.yellows = {i: [] for i in range(self.n_letters)}
        self.guessed_words = []
        self.all_words = all_words
        self.absent_letters = []
        self.present_letters = []
        self.correct_positions = [None] * self.n_letters # greens would be differnt per guess, remember correct positions
        self.state = None
        self.qstate = [0] * self.n_letters
        self.tmp_qstate = [0] * self.n_letters
        
    def try_word (self, guess):
        # need to add a check for not doing anything if the word is already guessed correctly
        guess = guess.upper()
        self.state = None
        self.current_yellows = [None]* self.n_letters # yellows have to reset each time
        self.greens = [None] * self.n_letters
        self.tmp_qstate = [0] * self.n_letters
        
        print('guessed: ', guess)
        if not len(guess) == self.n_letters:
            raise ValueError('wrong word length')
        if guess not in self.all_words:
            raise ValueError('invalid word')
        if guess in self.guessed_words:
            raise ValueError('word already guessed')
        self.guessed_words.append(guess)
        # c_g = character_guess,  c_w = character_word
        for i, (c_g, c_w) in enumerate(zip(guess, self.word)):
            if c_g == c_w:
                self.greens[i] = c_g
                self.correct_positions[i] = c_g
                self.qstate[i] = 1
                self.tmp_qstate[i] = 1
            # Check for number of non-None in greens list 
            if sum(bool(char) for char in self.greens) == self.n_letters:
                self.state = True
                print('victory! word is: ' + self.word) 
                grn_sum = sum([x!=None for x in self.greens])
                yel_sum = sum([x!=None for x in self.current_yellows])
                return grn_sum, yel_sum
            if c_g in self.word and c_g != c_w:
                self.yellows[i].append(c_g)
                self.current_yellows[i] = c_g
            if c_g in self.word: # track the letters that are there for the AI later
                self.present_letters.append(c_g)
            if c_g not in self.word: # track letters that are not there for the AI
                self.absent_letters.append(c_g)
        self.guess_counter -= 1
        grn_sum = sum([x!=None for x in self.greens])
        yel_sum = sum([x!=None for x in self.current_yellows])
        
        if self.guess_counter == 0:
            self.state=True
            print("YOU LOSE-TOO MANY GUESSES")
            return grn_sum, yel_sum # return number of greens and number of yellows
        print('Greens:' + str(self.greens) + 
              '\nYellows: ' + str(self.yellows) + 
              '\nGuesses left: ' + str(self.guess_counter))

        return grn_sum, yel_sum # return number of greens and number of yellows
        

In [284]:
w = Wordle(all_words = word_list)

In [289]:
GREENS = ['C', 'O', 'U', None, 'H']
all_words = word_list
match_list=[]
for i, letter in enumerate(GREENS):
    match_list+=[word for word in all_words if word[i]==letter]
    if letter is not None:
        print(i)
        all_words = list(match_list)
        match_list = []

0
1
2
4


In [290]:
print(match_list[0:5])
print(match_list[-5:])
print(len(match_list))

[]
[]
0


In [291]:
len(all_words)

2

In [292]:
all_words

['COUCH', 'COUGH']

In [299]:
# player agent class

# need to add the ability to take the correct letters as input to the guess methods


## this is where we can have different 'types' of word guesses to be optimized
class Player:
    def __init__(self, all_words):
        self.guesses=0
        self.guessed_words = []
        self.all_words = all_words
        self.tmp_wordlist = []
        
    def get_random_guess(self):
        '''
        make a guess randomly from the word list
        '''       
        current_guess = random.sample(set(self.all_words), 1)[0]
        while(current_guess in self.guessed_words):
            current_guess = random.sample(set(self.all_words), 1)[0]
        self.guessed_words.append(current_guess)
        return current_guess
    def get_common_guess(self):
        i=0
        while( self.all_words[i] in self.guessed_words):
            i+=1
        self.guessed_words.append(self.all_words[i])
        return self.all_words[i]
    
    def guess_by_green(self, GREENS):
        match_list=[]
        ALL_WORDS = self.all_words
        if not all(v is None for v in GREENS):
            for i, letter in enumerate(GREENS):
                match_list+=[word for word in ALL_WORDS if word[i]==letter]
                if letter is not None:
                    print(i)
                    ALL_WORDS = list(match_list)
                    match_list = []
            current_guess = random.sample(set(ALL_WORDS), 1)[0]
            # saw this fail on COUCH when it guessed COUGH
            while(current_guess in self.guessed_words):
                current_guess = random.sample(set(ALL_WORDS), 1)[0]
            self.guessed_words.append(current_guess)
            self.tmp_wordlist = ALL_WORDS
        else:
            current_guess = random.sample(set(self.all_words), 1)[0]
            while(current_guess in self.guessed_words):
                current_guess = random.sample(set(self.all_words), 1)[0]
            self.guessed_words.append(current_guess)
            self.tmp_wordlist = self.all_words
        return current_guess
    #def guess_by_probs(self): ## guess based on Ben's list


In [295]:
p = Player(word_list)

In [298]:
p.guess_by_green(['C', 'O', 'U', None, 'H'])

0
1
2
4


KeyboardInterrupt: 

In [146]:
e = Environment()

+++++++++++++++++++++++++++++++
the wordl is: JULIA
_______________________________


In [232]:
from gym import spaces

In [233]:
import gym
import itertools
import matplotlib
import matplotlib.style
import numpy as np
import pandas as pd
import sys
import gym_gridworlds
#env = gym.make('Gridworld-v0')
  
from collections import defaultdict

import plotting

In [264]:
## need an environment class that lets the player interact with wordl
class Environment:
    REWARD_YELLOW = 1
    REWARD_GREEN = 10
    REWARD_WIN = 100
    REWARD_LOSE = -500
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.game_over = False
        self.score = 0
        self.player = Player(all_words = word_list)
        self.wordle = Wordle(all_words=word_list)
        print('+++++++++++++++++++++++++++++++')
        print('the wordl is:', self.wordle.word)
        print('_______________________________')
        self.guesses = 0
        self.action_space = [0,1,2]
        self.greens = [None, None, None, None, None]
        self.tmp_wordlist = []
        
    def step(self, action):
        self.guesses += 1
        
        if action==0:
            current_guess = self.player.get_random_guess()
        elif action == 1:
            current_guess = self.player.get_common_guess()
        elif action == 2:
            current_guess = self.player.guess_by_green(self.wordle.greens)
        
        n_greens, n_yellows = self.wordle.try_word(current_guess)
        reward = 0
        
        if n_greens>0:
            reward += self.REWARD_GREEN * n_greens
            self.score +=self.REWARD_GREEN
        if n_yellows>0:
            reward += self.REWARD_YELLOW * n_yellows
            self.score +=self.REWARD_YELLOW * n_yellows
            
        if self.wordle.state == 'WIN':
            reward += self.REWARD_WIN
            self.score +=self.REWARD_WIN
            return state, reward, self.wordle.state
        
        result = self.wordle.tmp_qstate
        self.tmp_wordlist = self.player.tmp_wordlist
        return result, reward, self.wordle.state
        
        

In [265]:
e = Environment()

+++++++++++++++++++++++++++++++
the wordl is: SWANS
_______________________________


In [266]:
e.step(0)

guessed:  SEITZ
Greens:['S', None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: []}
Guesses left: 5


([1, 0, 0, 0, 0], 10, None)

In [237]:
#from stable_baselines.common.policies import MlpPolicy
#from stable_baselines.common.vec_env import DummyVecEnv
#from stable_baselines import PPO1

In [238]:
def createEpsilonGreedyPolicy(Q, epsilon, num_actions):
	"""
	Creates an epsilon-greedy policy based
	on a given Q-function and epsilon.
	
	Returns a function that takes the state
	as an input and returns the probabilities
	for each action in the form of a numpy array
	of length of the action space(set of possible actions).
	"""
	def policyFunction(state):

		Action_probabilities = np.ones(num_actions, dtype = float) * epsilon / num_actions
		best_action = np.argmax(Q[state])
		Action_probabilities[best_action] += (1.0 - epsilon)
		return Action_probabilities

	return policyFunction


In [239]:
e = Environment()

+++++++++++++++++++++++++++++++
the wordl is: DEWAR
_______________________________


In [240]:
e.action_space

[0, 1, 2]

In [241]:
import plottingepi

In [242]:
env = Environment()

+++++++++++++++++++++++++++++++
the wordl is: EMMET
_______________________________


In [243]:
env.action_space

[0, 1, 2]

In [300]:
def qLearning(env, num_episodes, discount_factor = 1.0, alpha = 0.5, epsilon = 0.3):
    """
    Q-Learning algorithm: Off-policy TD control.
    Finds the optimal greedy policy while improving
    following an epsilon-greedy policy"""
    
    # Action value function
    # A nested dictionary that maps
    # state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(len(env.action_space)))

    # Keeps track of useful statistics
    stats = plottingepi.EpisodeStats(
        episode_lengths = np.zeros(num_episodes),
        episode_rewards = np.zeros(num_episodes))
    
    # Create an epsilon greedy policy function
    # appropriately for environment action space
    policy = createEpsilonGreedyPolicy(Q, epsilon, len(env.action_space) )
    wordlistdict = defaultdict()
    
    # For every episode
    for ith_episode in range(num_episodes):

        # Reset the environment and pick the first action
        state = env.reset()
        
        for t in itertools.count():
            #print("this is q", Q)
            # get probabilities of all actions from current state
            action_probabilities = policy(state)
            print('action probabilities:', action_probabilities)
            # choose action according to
            # the probability distribution
            action = np.random.choice(np.arange(
                    len(action_probabilities)),
                    p = action_probabilities)
            print('this is the action', action)
            # take action and get reward, transit to next state
            next_state, reward, done = env.step(action)
            print(next_state)
            print('value of next state', str(sum(next_state)) )
            
            next_state = sum(next_state)

            # Update statistics
            stats.episode_rewards[ith_episode] += reward
            stats.episode_lengths[ith_episode] = t

            # TD Update
            best_next_action = np.argmax(Q[next_state])	
            td_target = reward + discount_factor * Q[next_state][best_next_action]
            td_delta = td_target - Q[state][action]
            Q[state][action] += alpha * td_delta

            # done is True if episode terminated
            if done:
                break

        state = next_state
        wordlistdict[env.wordle.word] = env.tmp_wordlist
            
    return Q, stats, wordlistdict


In [301]:
q1, s1, w1 = qLearning(Environment(), 100)

+++++++++++++++++++++++++++++++
the wordl is: LEGIS
_______________________________
+++++++++++++++++++++++++++++++
the wordl is: AVAIT
_______________________________
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  CIDER
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: ['I'], 2: [], 3: [], 4: []}
Guesses left: 5
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 2
guessed:  CLUNG
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: ['I'], 2: [], 3: [], 4: []}
Guesses left: 4
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  RAILS
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: ['I', 'A'], 2: ['I'], 3: [], 4: []}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  ABBAS
Greens:['A', None, None, None, None]
Yellows: {0: [], 1: ['I', 'A'], 2: ['I'], 3: ['A'], 4: []}
Guesses left: 2


1
4
guessed:  INMAN
YOU LOSE-TOO MANY GUESSES
[0, 1, 0, 0, 1]
value of next state 2
+++++++++++++++++++++++++++++++
the wordl is: FEEDS
_______________________________
action probabilities: [0.1 0.1 0.8]
this is the action 2
guessed:  EWELL
Greens:[None, None, 'E', None, None]
Yellows: {0: ['E'], 1: [], 2: [], 3: [], 4: []}
Guesses left: 5
[0, 0, 1, 0, 0]
value of next state 1
action probabilities: [0.1 0.1 0.8]
this is the action 2
2
guessed:  CRETE
Greens:[None, None, 'E', None, None]
Yellows: {0: ['E'], 1: [], 2: [], 3: [], 4: ['E']}
Guesses left: 4
[0, 0, 1, 0, 0]
value of next state 1
action probabilities: [0.1 0.1 0.8]
this is the action 0
guessed:  HOSES
Greens:[None, None, None, None, 'S']
Yellows: {0: ['E'], 1: [], 2: ['S'], 3: ['E'], 4: ['E']}
Guesses left: 3
[0, 0, 0, 0, 1]
value of next state 1
action probabilities: [0.1 0.1 0.8]
this is the action 2
4
guessed:  HIGGS
Greens:[None, None, None, None, 'S']
Yellows: {0: ['E'], 1: [], 2: ['S'], 3: ['E'], 4: ['E']}
Guesses left:

value of next state 0
action probabilities: [0.1 0.1 0.8]
this is the action 2
guessed:  BOXER
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: ['E'], 3: ['E'], 4: []}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.1 0.8 0.1]
this is the action 1
guessed:  THERE
Greens:[None, None, None, None, 'E']
Yellows: {0: [], 1: [], 2: ['E', 'E'], 3: ['E'], 4: []}
Guesses left: 2
[0, 0, 0, 0, 1]
value of next state 1
action probabilities: [0.1 0.8 0.1]
this is the action 0
guessed:  EXIST
Greens:[None, None, None, None, None]
Yellows: {0: ['E'], 1: [], 2: ['E', 'E'], 3: ['E'], 4: []}
Guesses left: 1
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.1 0.8 0.1]
this is the action 1
guessed:  WOULD
YOU LOSE-TOO MANY GUESSES
[1, 0, 0, 0, 0]
value of next state 1
+++++++++++++++++++++++++++++++
the wordl is: COUPS
_______________________________
action probabilities: [0.1 0.8 0.1]
this is the action 1
guessed:  WHICH
Greens:[None, None, None

guessed:  HOWLS
YOU LOSE-TOO MANY GUESSES
[0, 0, 0, 0, 0]
value of next state 0
+++++++++++++++++++++++++++++++
the wordl is: MINCE
_______________________________
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  CHATS
Greens:[None, None, None, None, None]
Yellows: {0: ['C'], 1: [], 2: [], 3: [], 4: []}
Guesses left: 5
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.1 0.1 0.8]
this is the action 2
guessed:  TALUS
Greens:[None, None, None, None, None]
Yellows: {0: ['C'], 1: [], 2: [], 3: [], 4: []}
Guesses left: 4
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  ABABA
Greens:[None, None, None, None, None]
Yellows: {0: ['C'], 1: [], 2: [], 3: [], 4: []}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.1 0.1 0.8]
this is the action 2
guessed:  MATER
Greens:['M', None, None, None, None]
Yellows: {0: ['C'], 1: [], 2: [], 3: ['E'], 4: []}
Guesses left: 2
[1, 0, 0, 0, 0]
valu

+++++++++++++++++++++++++++++++
the wordl is: MEALS
_______________________________
action probabilities: [0.8 0.1 0.1]
this is the action 2
guessed:  TUTTO
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: []}
Guesses left: 5
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  JEANS
Greens:[None, 'E', 'A', None, 'S']
Yellows: {0: [], 1: [], 2: [], 3: [], 4: []}
Guesses left: 4
[0, 1, 1, 0, 1]
value of next state 3
action probabilities: [0.8 0.1 0.1]
this is the action 1
guessed:  WHICH
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: []}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  SAITH
Greens:[None, None, None, None, None]
Yellows: {0: ['S'], 1: ['A'], 2: [], 3: [], 4: []}
Guesses left: 2
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 2
guessed:  TURKS


guessed:  TROPE
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: ['E']}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.1 0.1 0.8]
this is the action 2
guessed:  VERSA
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: ['E'], 2: [], 3: ['S'], 4: ['E']}
Guesses left: 2
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  DRURY
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: ['E'], 2: ['U'], 3: ['S'], 4: ['E']}
Guesses left: 1
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.1 0.1 0.8]
this is the action 2
guessed:  COMTE
YOU LOSE-TOO MANY GUESSES
[0, 0, 0, 0, 0]
value of next state 0
+++++++++++++++++++++++++++++++
the wordl is: SEVEN
_______________________________
action probabilities: [0.1 0.8 0.1]
this is the action 0
guessed:  NUEVA
Greens:[None, None, None, None, None]
Yellows: {0: ['N'], 1: [], 2: ['E'], 3: ['V'], 4: []}
Guesses left: 5
[

action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  FAURE
Greens:['F', 'A', None, None, 'E']
Yellows: {0: [], 1: [], 2: ['C'], 3: ['R'], 4: []}
Guesses left: 4
[1, 1, 0, 0, 1]
value of next state 3
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  ALORS
Greens:[None, None, None, None, None]
Yellows: {0: ['A'], 1: [], 2: ['C'], 3: ['R', 'R'], 4: []}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  PENCE
Greens:[None, None, None, 'C', 'E']
Yellows: {0: ['A'], 1: ['E'], 2: ['C'], 3: ['R', 'R'], 4: []}
Guesses left: 2
[0, 0, 0, 1, 1]
value of next state 2
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  KINGS
Greens:[None, None, None, None, None]
Yellows: {0: ['A'], 1: ['E'], 2: ['C'], 3: ['R', 'R'], 4: []}
Guesses left: 1
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 1
guessed:  WHICH
YOU LOSE-TOO MANY GUESSES
[0, 0, 0, 1,

action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  PANIC
Greens:['P', None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: []}
Guesses left: 4
[1, 0, 0, 0, 0]
value of next state 1
action probabilities: [0.8 0.1 0.1]
this is the action 1
guessed:  WHICH
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: []}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 2
guessed:  NANAK
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: []}
Guesses left: 2
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  FOYER
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: ['Y'], 3: ['E'], 4: []}
Guesses left: 1
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  FACES
YOU LOSE-TOO MANY GUESSES
[0, 0, 0, 0, 0]
value of next state 0
++++++++++++++++++

guessed:  STOOD
YOU LOSE-TOO MANY GUESSES
[0, 0, 0, 0, 0]
value of next state 0
+++++++++++++++++++++++++++++++
the wordl is: SWORN
_______________________________
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  TIDES
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: [], 4: ['S']}
Guesses left: 5
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 2
guessed:  AGENT
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: [], 3: ['N'], 4: ['S']}
Guesses left: 4
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  CYSTS
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: ['S'], 3: ['N'], 4: ['S', 'S']}
Guesses left: 3
[0, 0, 0, 0, 0]
value of next state 0
action probabilities: [0.8 0.1 0.1]
this is the action 0
guessed:  VIEUX
Greens:[None, None, None, None, None]
Yellows: {0: [], 1: [], 2: ['S'], 3: ['N'], 4: ['S', 'S']}
Guesses left:

In [287]:
w1.keys()

dict_keys(['VATER', 'SAWED', 'ANIMA', 'AMIDE', 'SOMME', 'IMPEL', 'FANON', 'CLICK', 'WENCH', 'LAITY', 'ILIUM', 'DAWNS', 'COUGH', 'DOVER', 'SLUMP', 'EXXON', 'AROMA', 'OPTIC', 'CLIVE', 'LISLE', 'WRITS', 'THIGH', 'YEMEN', 'YEAST', 'PARTI', 'NAVEL', 'PATNA', 'ADAPT', 'HALEY', 'ADIEU', 'TEDDY', 'EXITS', 'CLINE', 'ENEMA', 'MAYNE', 'BUILT', 'POLIS', 'CRUDE', 'LOGAN', 'CONAN', 'SWANS', 'DRIED', 'MINAS', 'SHAVE', 'DELVE', 'PANGS', 'RUNES', 'OVATE', 'ROYAL', 'FERRY', 'VAGAL', 'OCTET', 'SMELL', 'JUDGE', 'BASIC', 'BUELL', 'FOLDS', 'SPAWN', 'APNEA', 'QUIRK', 'SPURS', 'MAIDS', 'DORIA', 'CORAL', 'EASES', 'FAILS', 'DENSE', 'GANGS', 'GULLS', 'TIPSY', 'CAUSA', 'FOUNT', 'NEUER', 'TASTE', 'FOCUS', 'SAMOS', 'WILLI', 'ROUGE', 'LENOX', 'JAKOB', 'SANTA', 'DONOR', 'LOADS', 'BLAZE', 'PLUGS', 'ULAMA', 'SINDH', 'ABOUT', 'SOUND', 'CRAVE', 'UNTER', 'FATTY', 'RYDER', 'CLEVE', 'BONNY', 'CRANK', 'CASES', 'DRAIN', 'CLIME', 'LOIRE'])

In [288]:
w1['COUGH']

['WHICH',
 'THEIR',
 'THERE',
 'WOULD',
 'OTHER',
 'THESE',
 'ABOUT',
 'FIRST',
 'COULD',
 'AFTER',
 'THOSE',
 'WHERE',
 'BEING',
 'UNDER',
 'YEARS',
 'GREAT',
 'STATE',
 'WORLD',
 'THREE',
 'WHILE',
 'FOUND',
 'MIGHT',
 'STILL',
 'RIGHT',
 'PLACE',
 'EVERY',
 'POWER',
 'SINCE',
 'GIVEN',
 'NEVER',
 'ORDER',
 'WATER',
 'SMALL',
 'SHALL',
 'LARGE',
 'POINT',
 'AGAIN',
 'OFTEN',
 'AMONG',
 'HOUSE',
 'WOMEN',
 'GROUP',
 'THINK',
 'HUMAN',
 'LATER',
 'UNTIL',
 'WHOLE',
 'EARLY',
 'MEANS',
 'ABOVE',
 'VALUE',
 'STUDY',
 'TABLE',
 'TAKEN',
 'TIMES',
 'KNOWN',
 'COURT',
 'YOUNG',
 'WORDS',
 'WHITE',
 'LIGHT',
 'LEAST',
 'LEVEL',
 'CHILD',
 'DEATH',
 'PRESS',
 'CASES',
 'GOING',
 'PARTY',
 'USING',
 'SENSE',
 'BASED',
 'WHOSE',
 'SOUTH',
 'TOTAL',
 'CLASS',
 'LOCAL',
 'ALONG',
 'TERMS',
 'MONEY',
 'BLACK',
 'FORCE',
 'NORTH',
 'NIGHT',
 'SHORT',
 'FIELD',
 'ASKED',
 'QUITE',
 'THING',
 'WOMAN',
 'MAJOR',
 'THIRD',
 'SHOWN',
 'BEGAN',
 'CAUSE',
 'HEART',
 'SEEMS',
 'TRADE',
 'CLEAR',
 'MODEL',
